# Credit Scoring

In [ ]:
from pandas import Series
import pandas as pd
import numpy as np
from datetime import datetime
import re


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve

## DATA

In [ ]:
DATA_DIR = '/kaggle/input/scoring-ring/'
df_train = pd.read_csv(DATA_DIR+'/train.csv')
df_test = pd.read_csv(DATA_DIR+'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_test.info()

In [ ]:
sample_submission.info()

In [ ]:
# дря корректной обработки признаков объединяем трейн и тест в один датасет

df_train['sample'] = 1 # помечаем трейн
df_test['sample'] = 0 # помечаем тест
df_test['default'] = 0 # в тесте нет значения default, его нужно предсказать, по этому пока заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем тест и трейн

In [ ]:
data.info()

## Cleaning and Prepping Data

Среди признаков те, что уже рассматривались в предыдущих заданиях:
* client_id - идентификатор клиента
* education - уровень образования
* sex - пол заёмщика
* age - возраст заёмщика
* car - флаг наличия автомобиля
* car_type - флаг автомобиля-иномарки
* decline_app_cnt - количество отказанных прошлых заявок
* good_work - флаг наличия «хорошей» работы
* bki_request_cnt - количество запросов в БКИ
* home_address - категоризатор домашнего адреса
* work_address - категоризатор рабочего адреса
* income - доход заёмщика
* foreign_passport - наличие загранпаспорта
* default - наличие дефолта

А также присутствуют новые признаки:
* app_date - дата подачи заявки
* score_bki - скоринговый балл по данным из БКИ
* region_rating - рейтинг региона 
* sna - связь заемщика с клиентами банка  
* first_time - давность наличия информации о заемщике


## education

Среди всех признаков только education имеет пропуски (478): 

In [ ]:
data['education'].isna().sum()

In [ ]:
data['education'].value_counts()

In [ ]:
data['education'].value_counts().plot.barh()

Наличие пропусков в этом признаке может свидетельствовать о том, что во время заполнения анкеты это поле является необязательным и может быть пропущено. 
Пропуски заменим на новое значение 'N/A'. Также этот признак можно отнести к категориальным и применить к нему get_dummies.

In [ ]:
data['education'] = data['education'].apply(lambda x: 'N/A' if pd.isna(x) else x)

In [ ]:
data[data['sample']==1].pivot_table(index=['education'], values='default', aggfunc='mean', fill_value=0).sort_values(['default']) #Чем больше mean, тем больше дефолтов

Ализировав полученной таблицы: значение mean для признака N/A находится между значениями mean для GRD и UGR, и значительно отличается от значения mean для наиболее часто втречающегося признака SCH. Таким образом заполнение пропусков новым значением N/A более корректное, чем заполнение значением SCH



## app_date

In [ ]:
data['app_date']

In [ ]:
data['app_date'].value_counts()

In [ ]:
# Переведем значения данного признака во временной формат
data['app_date'] = data['app_date'].apply(lambda x: datetime.strptime(x, '%d%b%Y'))

In [ ]:
data['app_date'].value_counts()

Все наблюдения датированы в промежутке c январь по апрель 2014 года

In [ ]:
year = data['app_date'].apply(lambda x: x.year)
year.value_counts()

In [ ]:
month = data['app_date'].apply(lambda x: x.month).rename('month', inplace=True)

month.value_counts()

In [ ]:
weekday = data['app_date'].apply(lambda x: x.weekday()).rename('weekday', inplace=True)

weekday.value_counts()

In [ ]:
day = data['app_date'].apply(lambda x: x.day).rename('day', inplace=True)

day.value_counts()

In [ ]:
# Добавим дополнительные признаки месяц и день недели подачи заявки и :
data_proc = pd.concat([data, month, weekday, day], axis=1)




In [ ]:
data_proc[data_proc['sample']==1]['month']

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['month'], values='default', aggfunc='mean', fill_value=0).sort_values(['default']) #Чем больше mean, тем больше дефолтов

In [ ]:
plt.figure()
sns.distplot(data_proc['month'][(data_proc['default'] == 0) & (data_proc['sample'] == 1)], hist=True, kde = True, rug=False, bins=4)
sns.distplot(data_proc['month'][data_proc['default'] == 1], hist=True, kde = True, rug=False, bins=4)
plt.title('month')
#plt.xlim(0,100000)
plt.show()

In [ ]:
sns.boxplot(y='month', x='default', hue='default', data=data_proc[data_proc['sample']==1], palette="Set3")

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['weekday'], values='default', aggfunc='mean', fill_value=0).sort_values(['default']) #Чем больше mean, тем больше дефолтов

In [ ]:
plt.figure()
sns.distplot(data_proc['weekday'][data['default'] == 0], hist=True, kde = True, rug=False, bins=7)
sns.distplot(data_proc['weekday'][data['default'] == 1], hist=True, kde = True, rug=False, bins=7)
plt.title('weekday')
#plt.xlim(0,100000)
plt.show()

In [ ]:
sns.boxplot(y='weekday', x='default', hue='default', data=data_proc[data_proc['sample']==1], palette="Set3")

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['day'], values='default', aggfunc='mean', fill_value=0).sort_values(['default']) #Чем больше mean, тем больше дефолтов

In [ ]:
plt.figure()
sns.distplot(data_proc['day'][data['default'] == 0], hist=True, kde = True, rug=False, bins=30)
sns.distplot(data_proc['day'][data['default'] == 1], hist=True, kde = True, rug=False, bins=30)
plt.title('day')
#plt.xlim(0,100000)
plt.show()

In [ ]:
sns.boxplot(y='day', x='default', hue='default', data=data_proc[data_proc['sample']==1], palette="Set3")

Вывод: количество дефолтов скорее не зависит или слабо зависит от дня месяца или недели. При рассмотрении по месяцам, кажется, что в первый месяц дефолтов было больше, потом их количество снижалось. Этот признак можно будет использовать для построения модели.

## sex

In [ ]:
data['sex'].value_counts()

In [ ]:
# Преобразуем значения в числа с помощью LabelEncoder
label_encoder = LabelEncoder()
data_proc['sex'] = label_encoder.fit_transform(data['sex'])
print('sex', dict(enumerate(label_encoder.classes_)))

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['sex'], values='default', aggfunc='mean', fill_value=0).sort_values(['default']) #Чем больше mean, тем больше дефолтов

In [ ]:
plt.figure()
sns.distplot(data_proc['sex'][data['default'] == 0], hist=True, kde = True, rug=False, bins=2)
sns.distplot(data_proc['sex'][data['default'] == 1], hist=True, kde = True, rug=False, bins=2)
plt.title('sex')
#plt.xlim(0,100000)
plt.show()

## age

In [ ]:
plt.figure()
sns.distplot(data_proc['age'], hist=True, kde = False, rug=False)
plt.title('age')
plt.show()

In [ ]:
sns.boxplot(y='age', x='default', hue='default', data=data_proc[data_proc['sample']==1], palette="Set3")

In [ ]:
plt.figure()
sns.distplot(data_proc['age'][(data_proc['default'] == 0)  & (data_proc['sample']==1)], hist=True, kde = True, rug=False, bins=35)
sns.distplot(data_proc['age'][data_proc['default'] == 1], hist=True, kde = True, rug=False, bins=35)
plt.title('age')
#plt.xlim(0,100000)
plt.show()

Видно, что дефолты происходят немного чаще у людей более молодого возраста. Добавим новый призна 'age_40', который равен 1, если возраст > 40, и 0, если возраст < 40.

In [ ]:
data_proc['age']

In [ ]:
data_proc['age_40'] = data_proc['age'].apply(lambda x: 1 if 40<=x else 0)

In [ ]:
plt.figure()
sns.distplot(data_proc['age_40'][data['default'] == 0  & (data_proc['sample']==1)], hist=True, kde = True, rug=False, bins=35)
sns.distplot(data_proc['age_40'][data['default'] == 1], hist=True, kde = True, rug=False, bins=35)
plt.title('age_40')
#plt.xlim(0,100000)
plt.show()

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['age_40'], values='default', aggfunc='mean', fill_value=0).sort_values(['default']) #Чем больше mean, тем больше дефолтов

## car

In [ ]:
data['car'].value_counts()

In [ ]:
# Чтобы выразить значения бинарного признака в форме чисел, используем LabelEncoder
label_encoder = LabelEncoder()
data_proc['car'] = label_encoder.fit_transform(data_proc['car'])
print('car', dict(enumerate(label_encoder.classes_)))

In [ ]:
plt.figure()
sns.distplot(data_proc['car'][(data_proc['default'] == 0) & (data_proc['sample']==1)], hist=True, kde = True, rug=False, bins=2)
sns.distplot(data_proc['car'][data_proc['default'] == 1], hist=True, kde = True, rug=False, bins=2)
plt.title('car')
#plt.xlim(0,100000)
plt.show()

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['car'], values='default', aggfunc='mean', fill_value=0).sort_values(['default']) #Чем больше mean, тем больше дефолтов

Случаев дефолтов меньше у людей с машиной

## car_type

In [ ]:
data['car_type'].value_counts()

In [ ]:
data_proc['car_type'] = label_encoder.fit_transform(data_proc['car_type'])
print('car_type', dict(enumerate(label_encoder.classes_)))

In [ ]:
plt.figure()
sns.distplot(data_proc['car_type'][(data_proc['default'] == 0)  & (data_proc['sample']==1)], hist=True, kde = True, rug=False, bins=2)
sns.distplot(data_proc['car_type'][data_proc['default'] == 1], hist=True, kde = True, rug=False, bins=2)
plt.title('car_type')
#plt.xlim(0,100000)
plt.show()

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['car_type'], values='default', aggfunc='mean', fill_value=0).sort_values(['default']) #Чем больше mean, тем больше дефолтов

In [ ]:
#Добавим признак 'car_type_all', который объединяет данные по машине: 0 - машины нет, 1 - машина не иномарка, 2 - машина - иномарка
data_proc['car_all'] = data_proc['car'] + data_proc['car_type']

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['car_all'], values='default', aggfunc='mean', fill_value=0).sort_values(['default']) #Чем больше mean, тем больше дефолтов

Видно, что количество дефолтов у клиентов без машины и с отечественным автомобилем очень близки.

## decline_app_cnt

In [ ]:
data['decline_app_cnt'].value_counts()

In [ ]:
plt.figure()
sns.distplot(data_proc['decline_app_cnt'][(data_proc['default'] == 0)  & (data_proc['sample']==1)], hist=True, kde = True, rug=False, bins=2)
sns.distplot(data_proc['decline_app_cnt'][data_proc['default'] == 1], hist=True, kde = True, rug=False, bins=2)
plt.title('decline_app_cnt')
#plt.xlim(0,100000)
plt.show()

In [ ]:
ax= sns.boxplot(y='decline_app_cnt', x='default', hue='default', data=data_proc[data_proc['sample']==1], palette="Set3")
ax.set_ylim(0,5)

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['decline_app_cnt'], values='default', aggfunc='mean', fill_value=0).sort_values(['decline_app_cnt']) #Чем больше mean, тем больше дефолтов

В целом видна тенденция на увеличение количества дефолтов с увеличением числа отклонений, после 2 отклонений, значений слишком мало, чтобы оценить тенденцию, поэтому заменим все значения больше 2 на 2. что будет означать все случаи с числом отклонений более 2 раз. *Тестировались различные значения для замены, но в случае "2" количество выбросов было минимальным

In [ ]:
data_proc['decline_app_cnt'] = data_proc['decline_app_cnt'].apply(lambda x: 2 if x>2 else x)

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['decline_app_cnt'], values='default', aggfunc='mean', fill_value=0).sort_values(['decline_app_cnt']) #Чем больше mean, тем больше дефолтов

In [ ]:
data_proc[data_proc['sample'] == 1]['decline_app_cnt'].value_counts()

In [ ]:
sns.boxplot(x='default', y= 'decline_app_cnt', data=data_proc[data_proc['sample'] == 1])

## good_work

In [ ]:
data['good_work'].value_counts()

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['good_work'], values='default', aggfunc='mean', fill_value=0).sort_values(['default']) #Чем больше mean, тем больше дефолтов

## score_bki

In [ ]:
data['score_bki'].value_counts()

In [ ]:
sns.boxplot(x='default', y= 'score_bki', data=data)
#ax.set(ylim=(0, 5))

In [ ]:
data['score_bki'].max()

In [ ]:
data['score_bki'].min()

In [ ]:
plt.figure()
sns.distplot(data['score_bki'][(data['default'] == 0) & (data['sample'] == 1)], hist=True, kde = True, rug=False)
sns.distplot(data['score_bki'][data['default'] == 1], hist=True, kde = True, rug=False)
plt.title('score_bki')
plt.show()

Значения признака лежат в промежутке от -3.7 до 0.2. Такие значения скорингового балла не соответствуют ни одной из скоринговых систем. В то же время признак распределен нормально, без явных выбросов. 

## bki_request_cnt

In [ ]:
data['bki_request_cnt'].value_counts()

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['bki_request_cnt'], values='default', aggfunc='mean', fill_value=0).sort_values(['bki_request_cnt']) #Чем больше mean, тем больше дефолтовA

Здесь также можно выделить тенденцию на увеличение дефолтов с увеличением количества запросов. Чтобы выделить тенденцию в явном виде, произведем замену значений: 7 - если запросов от 7 до 9, и 8 - если запросов 10 и больше

In [ ]:
data_proc['bki_request_cnt'] = data['bki_request_cnt'].apply(lambda x: 7 if x in [7, 8, 9] else x).apply(lambda x: 8 if x >= 10 else x)

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['bki_request_cnt'], values='default', aggfunc='mean', fill_value=0).sort_values(['bki_request_cnt']) #Чем больше mean, тем больше дефолтов

In [ ]:
data_proc['bki_request_cnt'].value_counts()

In [ ]:
sns.boxplot(x='default', y= 'bki_request_cnt', data=data_proc[data_proc['sample']==1])

## region_rating

In [ ]:
data['region_rating'].value_counts()

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['region_rating'], values='default', aggfunc='mean', fill_value=0).sort_values(['region_rating']) #Чем больше mean, тем больше дефолтов

Здесь также видна тенденция на сокращение количества дефолтов с увеличением рейтинга региона

In [ ]:
sns.boxplot(x='default', y= 'region_rating', data=data_proc[data_proc['sample']==1])

## home_address

In [ ]:
data['home_address'].value_counts()

In [ ]:
data_proc[data_proc['sample']==1].pivot_table(index=['home_address'], values='default', aggfunc='mean', fill_value=0).sort_values(['home_address']) #Чем больше mean, тем больше дефолтов

In [ ]:
plt.figure()
sns.distplot(data['home_address'][(data['default'] == 0) & (data['sample'] == 1)], hist=True, kde = True, rug=False)
sns.distplot(data['home_address'][data['default'] == 1], hist=True, kde = True, rug=False)
plt.title('home_address')
plt.show()

## work_address

In [ ]:
data['work_address'].value_counts()

In [ ]:
data[data['sample']==1].pivot_table(index=['work_address'], values='default', aggfunc='mean', fill_value=0).sort_values(['work_address']) #Чем больше mean, тем больше дефолтов

In [ ]:
plt.figure()
sns.distplot(data['work_address'][(data['default'] == 0) & (data['sample'] == 1)], hist=True, kde = True, rug=False)
sns.distplot(data['work_address'][data['default'] == 1], hist=True, kde = True, rug=False)
plt.title('work_address')
plt.show()

## income

In [ ]:
data['income'].value_counts()

In [ ]:
plt.figure()
sns.distplot(data['income'][(data['default'] == 0) & (data['sample'] == 1)], hist=True, kde = True, rug=False)
sns.distplot(data['income'][data['default'] == 1], hist=True, kde = True, rug=False)
plt.title('income')
plt.xlim(0,100000)
plt.show()

In [ ]:
data['income'].min()

In [ ]:
data['income'].max()

In [ ]:
sns.boxplot(x='default', y= 'income', data=data[data['sample'] == 1])
#ax.set(ylim=(0, 5))

Здесь много выбросов. Тем не менее заметно, что чем больше зарплата, тем меньше дефотов. Создадим новый признак, который показывает порядок зарплаты:

In [ ]:
data_proc['income_rank'] = data['income'].apply(lambda x: round(np.log10(x)))

In [ ]:
data_proc['income_rank'].value_counts()

In [ ]:
data_proc[data['sample']==1].pivot_table(index=['income_rank'], values='default', aggfunc='mean', fill_value=0).sort_values(['income_rank']) #Чем больше mean, тем больше дефолтов

Получилось, что при доходе уровня "3" - дефолтов не наблюдалось, но это может быть вызвано в том числе и из-за небольшого количества случаев. Объединим их с уровнем дохода "4". Для уровней дохода "5" и "6" разница маленькая, объединим и их тоже.

In [ ]:
data_proc['income_rank'] = data['income'].apply(lambda x: 0 if round(np.log10(x))<=4 else 1)

In [ ]:
data_proc[data['sample']==1].pivot_table(index=['income_rank'], values='default', aggfunc='mean', fill_value=0).sort_values(['income_rank']) #Чем больше mean, тем больше дефолтов

## sna

In [ ]:
data['sna'].value_counts()

In [ ]:
data_proc[data['sample']==1].pivot_table(index=['sna'], values='default', aggfunc='mean', fill_value=0).sort_values(['sna']) #Чем больше mean, тем больше дефолтов

## first_time

In [ ]:
data['first_time'].value_counts()

In [ ]:
data_proc[data['sample']==1].pivot_table(index=['first_time'], values='default', aggfunc='mean', fill_value=0).sort_values(['first_time']) #Чем больше mean, тем больше дефолтов

## foreign_passport

In [ ]:
data['foreign_passport'].value_counts()

In [ ]:
data_proc['foreign_passport'] = label_encoder.fit_transform(data_proc['foreign_passport'])
print('foreign_passport', dict(enumerate(label_encoder.classes_)))

In [ ]:
data_proc[data['sample']==1].pivot_table(index=['foreign_passport'], values='default', aggfunc='mean', fill_value=0).sort_values(['foreign_passport']) #Чем больше mean, тем больше дефолтов

## default 

Проверим, достаточно ли образцов с каждым значением default. В этом случае используем df_train, так как только в этой таблице есть оригинальные значения.

In [ ]:
df_train['default'].value_counts()

In [ ]:
plt.figure()
sns.distplot(df_train['default'], hist=True, kde = False, rug=False, bins=2)
plt.title('default')
plt.show()

Количество случаев с положительным исходом составляет ~1/7 от всех случаев. Таким образом делаем вывод, что случаев обоих исходов достаточное количество.

# Data Preprocessing

In [ ]:
DATA_DIR = '/kaggle/input/scoring-ring/'
df_train = pd.read_csv(DATA_DIR+'/train.csv')
df_test = pd.read_csv(DATA_DIR+'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')
df_train['sample'] = 1 # помечаем трейн
df_test['sample'] = 0 # помечаем тест
df_test['default'] = 0 # в тесте нет значения default, его нужно предсказать, по этому пока заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем тест и трейн

data.info()

In [ ]:
del_col = ['app_date'] 

In [ ]:
def preproc_data(df_input):
    
    df_output = df_input.copy()
    
    '''Заполнение пропусков в признаке education'''
    df_output['education'] = df_output['education'].apply(lambda x: 'N/A' if pd.isna(x) else x)
    
    '''Преобразование значений признака app_date во временной формат'''
    df_output['app_date'] = df_output['app_date'].apply(lambda x: datetime.strptime(x, '%d%b%Y'))
    
    '''Новый признак month - месяц подачи заявки'''
    df_output['month'] = df_output['app_date'].apply(lambda x: x.month).rename('month', inplace=True)
    
    '''Новый признак age_40 - бинарный признак'''
    df_output['age_40'] = df_output['age'].apply(lambda x: 1 if 40<=x else 0)
    
    '''Преобразоваие значений признака decline_app_cnt - категориальный признак'''
    df_output['decline_app_cnt'] = df_output['decline_app_cnt'].apply(lambda x: 2 if x>2 else x)
    
    '''Преобразоваие значений признака bki_request_cnt - категориальный признак'''
    df_output['bki_request_cnt'] = df_output['bki_request_cnt'].apply(lambda x: 7 if x in [7, 8, 9] else x).apply(lambda x: 8 if x >= 10 else x)
    
    '''Новый признак income_rank - бинарный признак'''
    df_output['income_rank'] = df_output['income'].apply(lambda x: 0 if round(np.log10(x))<=4 else 1)
    
    '''Список числовых признаков'''
    num_cols = ['age', 'score_bki', 'income']
    
    '''Список категориальных признаков'''
    cat_cols = ['education', 'decline_app_cnt', 'bki_request_cnt', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time', 'month']
    
    '''Список бинарных признаков'''
    bin_cols = ['sex', 'car', 'car_type', 'good_work', 'income_rank', 'foreign_passport']
    
    '''Стандартизация числовых признаков'''
    for col in num_cols:
        df_output[col] = StandardScaler().fit_transform(df_output[[col]].values)
  
    '''Генерация dummy-переменных для категориальных признаков'''
    for col in cat_cols:
        df_output = pd.get_dummies(df_output, columns=[col])
    
    '''Приведение бинарных признаков к числовому формату'''
    for col in bin_cols:
        df_output[col] = LabelEncoder().fit_transform(df_output[col])
    
    return df_output

In [ ]:
df_preproc = preproc_data(data)

In [ ]:
df_preproc.head(10)

In [ ]:
df_preproc.info()

In [ ]:
df_preproc.drop(['app_date'], axis = 1, inplace=True)

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['client_id','sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['client_id','sample'], axis=1)

y = train_data['default'].values            # наш таргет
X = train_data.drop(['default'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
probs = model.predict_proba(X_test)
probs = probs[:,1]


fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV

# Добавим типы регуляризации  penalty = ['l1', 'l2']
penalty = ['l1','l2']

# Зададим ограничения для параметра регуляризации
C = np.logspace(0, 4, 10)

# Создадим гиперпараметры
hyperparameters = dict(C=C, penalty=['l2'])

# Создаем сетку поиска с использованием 5-кратной перекрестной проверки
clf = GridSearchCV(model, hyperparameters, cv=20, verbose=0)
best_model = clf.fit(X_train, y_train)

# View best hyperparameters
print('Лучшее Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Лучшее C:', best_model.best_estimator_.get_params()['C'])

probs = best_model.predict_proba(X_test)
probs = probs[:,1]
print('roc_auc:', roc_auc_score(y_test, probs))

In [ ]:
probs = best_model.predict_proba(X_test)
probs = probs[:,1]


fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
# Используем класс LogisticRegressionCV.
from sklearn.linear_model import LogisticRegressionCV

# Сначала для значения penalty = 'l1'
for solver in ['saga', 'liblinear']:
    clf = LogisticRegressionCV(cv=5, random_state=0, verbose=0, penalty='l1', solver=solver).fit(X_train, y_train)
    probs = clf.predict_proba(X_test)
    probs = probs[:,1]
    print ("penalty = 'l1', solver = '", solver, "'roc_auc = ", roc_auc_score(y_test, probs))

In [ ]:
# Теперь для значения penalty = 'l2'
for solver in ['newton-cg', 'lbfgs', 'sag']:
    clf = LogisticRegressionCV(cv=5, random_state=0, verbose=0, penalty='l2', solver=solver).fit(X_train, y_train)
    best_model = clf.fit(X_train, y_train)
    probs = clf.predict_proba(X_test)
    probs = probs[:,1]
    print ("penalty = 'l2', solver = '", solver, "'roc_auc = ", roc_auc_score(y_test, probs))


In [ ]:
clf = LogisticRegressionCV(cv=5, random_state=0, verbose=0, penalty='elasticnet', solver='saga', l1_ratios=[0.5]).fit(X_train, y_train)
probs = clf.predict_proba(X_test)
probs = probs[:,1]
print ("penalty = 'elasticnet', solver = 'saga', 'roc_auc = ", roc_auc_score(y_test, probs))

In [ ]:
best_model = LogisticRegressionCV(cv=5, random_state=0).fit(X_train, y_train)

probs = best_model.predict_proba(X_test)
probs = probs[:,1]


fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

In [ ]:
best_model.get_params()

# Submission

In [ ]:
test_data = df_preproc.query('sample == 0').drop(['sample','default'], axis=1)

In [ ]:
test_data.info()

In [ ]:
best_model = GridSearchCV(model, hyperparameters, cv=20, verbose=0).fit(X_train, y_train)
y_pred = best_model.predict_proba(test_data.drop(['client_id'], axis=1))
test_data['default'] = y_pred[:, 1]
sample_submission = test_data[['client_id','default']]
sample_submission

In [ ]:
sample_submission.to_csv('sample_submission.csv', index = False)